# GIS Hotels Data Enrichment

- **Hotel buffer**

- **Hotels Airbnb around each hotel**
    - Total number of airbnbs
    - Average price of Airbnbs
    - Total number of airbnb beds (accomodates field)

- **Hotels Housing**
    - Total number of houses sold during last year
    - Median price of the house during last year

- **Hotels restaurants**
    - Total of restaurants
    - Total of reviews of restaurants (popularity of the area)
    - Average ranking
    
- **Acccessibility**
    - Airport combined travel time (min) to: Heathrow, Gatwick, Stanstead, Luton 
    - Combined Attractions travel time (min) to 10 locations of interest (business and tourist). 
    

In [16]:
import arcpy
arcpy.env.overwriteOutput = True

### Hotel Buffer

In [25]:
#arcpy.env.workspace = "C:/data"
arcpy.env.workspace = './Data'

#read the shapefile
hotels_shp = './Data/Shapefiles/hotels.shp'
arcpy.Buffer_analysis(hotels_shp, "./Data/Shapefiles/hotels_buffer", "500 Meters")

<Result '.\\Data\\Shapefiles\\hotels_buffer.shp'>

### Hotel Airbnb

In [27]:
#read the shapefiles
arcpy.env.workspace = './Data/Shapefiles'
hotels_buffer = 'hotels_buffer.shp'
airbnb = 'airbnb.shp'

#Field mappings for the different metrics we want to get.
#More info here = 'https://pro.arcgis.com/es/pro-app/arcpy/classes/fieldmap.htm'
#and here! : https://gis.stackexchange.com/questions/216391/setting-list-of-fields-to-field-mapping-of-spatial-join-using-arcpy

#Field map with all outputs ,first we create the object
fieldmappings = arcpy.FieldMappings()

#we add all field mappings, by default the calculation is First so need to change to Median if that is what we want.
fieldmappings.addTable(airbnb)
fieldmappings.addTable(hotels_buffer)
#this is how fieldmappings looks like for each variable 'Field1 "Field1" true true false 10 Long 0 10,First,#.....' 

#print(fieldmappings)

#From the Airbnb table we jsut neeed:
    #The Count (which is done automatically)
    #Median Price
    #Total accomodates


interested_fields = ['index','price','accommodat']
fields_to_delete = [f.name for f in fieldmappings.fields if f.name not in interested_fields]
for field in fields_to_delete:
    fieldmappings.removeFieldMap(fieldmappings.findFieldMapIndex(field))
    
    
    
#We do this manually,first we print fieldmappings and  copy paste the string 
#and change the different 'First' for the calculation we need.
fieldmappings = 'index "index" true true false 10 Long 0 10,First,#,./Data/Shapefiles\hotels_buffer.shp,index,-1,-1;accommodat "accommodat" true true false 10 Long 0 10,Sum,#,./Data/Shapefiles\airbnb.shp,accommodat,-1,-1;price "price" true true false 10 Long 0 10,Median,#,./Data/Shapefiles\airbnb.shp,price,-1,-1' 

####
# Spatial join with Hotels
####

arcpy.SpatialJoin_analysis(target_features=hotels_buffer, join_features=airbnb,
                           out_feature_class=r'./Data/Shapefiles/hotels_airbnb.shp',
                           join_operation='JOIN_ONE_TO_ONE',join_type='KEEP_ALL',
                           field_mapping=fieldmappings,match_option='COMPLETELY_CONTAINS',
                           search_radius=None,distance_field_name=None)


<Result '.\\Data\\Shapefiles\\hotels_airbnb.shp'>

### Hotels Housing

In [29]:
#read the shapefiles
arcpy.env.workspace = './Data/Shapefiles'
hotels_buffer = 'hotels_buffer.shp'
housing = 'house_transactions.shp'

#Field map with all outputs ,first we create the object
fieldmappings = arcpy.FieldMappings()

#we add all field mappings, by default the calculation is First so need to change to Median if that is what we want.
fieldmappings.addTable(housing)
fieldmappings.addTable(hotels_buffer)
#this is how fieldmappings looks like for each variable 'Field1 "Field1" true true false 10 Long 0 10,First,#.....' 

#print(fieldmappings)

#From the Airbnb table we jsut neeed:
    #The Count (which is done automatically)
    #Median Price


interested_fields = ['index','Price']
fields_to_delete = [f.name for f in fieldmappings.fields if f.name not in interested_fields]
for field in fields_to_delete:
    fieldmappings.removeFieldMap(fieldmappings.findFieldMapIndex(field))
    
    
    
#We do this manually,first we print fieldmappings and  copy paste the string 
#and change the different 'First' for the calculation we need.
fieldmappings = 'index "index" true true false 10 Long 0 10,First,#,./Data/Shapefiles\hotels_buffer.shp,index,-1,-1;Price "Price" true true false 10 Long 0 10,Median,#,./Data/Shapefiles\house_transactions.shp,Price,-1,-1' 

####
# Spatial join with Hotels
####

arcpy.SpatialJoin_analysis(target_features=hotels_buffer, join_features=housing,
                           out_feature_class=r'./Data/Shapefiles/hotels_housing.shp',
                           join_operation='JOIN_ONE_TO_ONE',join_type='KEEP_ALL',
                           field_mapping=fieldmappings,match_option='COMPLETELY_CONTAINS',
                           search_radius=None,distance_field_name=None)


<Result '.\\Data\\Shapefiles\\hotels_housing.shp'>

### Hotels Restaurants

In [32]:
#read the shapefiles
arcpy.env.workspace = './Data/Shapefiles'
hotels_buffer = 'hotels_buffer.shp'
restaurants = 'restaurants.shp'

#Field map with all outputs ,first we create the object
fieldmappings = arcpy.FieldMappings()

#we add all field mappings, by default the calculation is First so need to change to Median if that is what we want.
fieldmappings.addTable(restaurants)
fieldmappings.addTable(hotels_buffer)
#this is how fieldmappings looks like for each variable 'Field1 "Field1" true true false 10 Long 0 10,First,#.....' 

#print(fieldmappings)

#From the Airbnb table we jsut neeed:
    #The Count (which is done automatically)
    #Total sum of reviews (popularity of the area)
    #Average (mean) ranking (sentiment of the area)


interested_fields = ['index','Number_of', 'Ranking']
fields_to_delete = [f.name for f in fieldmappings.fields if f.name not in interested_fields]
for field in fields_to_delete:
    fieldmappings.removeFieldMap(fieldmappings.findFieldMapIndex(field))
    
    
    
#We do this manually,first we print fieldmappings and  copy paste the string 
#and change the different 'First' for the calculation we need.
fieldmappings = 'index "index" true true false 10 Long 0 10,First,#,./Data/Shapefiles\hotels_buffer.shp,index,-1,-1;Number_of "Number_of" true true false 10 Long 0 10,Sum,#,./Data/Shapefiles\restaurants.shp,Number_of,-1,-1;Ranking "Ranking" true true false 19 Double 0 0,Mean,#,./Data/Shapefiles\restaurants.shp,Ranking,-1,-1;' 

####
# Spatial join with Hotels
####

arcpy.SpatialJoin_analysis(target_features=hotels_buffer, join_features=restaurants,
                           out_feature_class=r'./Data/Shapefiles/hotels_restaurants.shp',
                           join_operation='JOIN_ONE_TO_ONE',join_type='KEEP_ALL',
                           field_mapping=fieldmappings,match_option='COMPLETELY_CONTAINS',
                           search_radius=None,distance_field_name=None)


<Result '.\\Data\\Shapefiles\\hotels_restaurants.shp'>